<a href="https://colab.research.google.com/github/gacerioni/redis-workshop-series-ptbr-gabs/blob/main/redis_workshop_alem_do_cache_nov_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop - Redis muito al√©m do Cache - 2024

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)



Bem-vindo ao workshop hands-on "Redis Muito Al√©m do Cache"! Aqui, voc√™ ter√° uma experi√™ncia pr√°tica com os principais data types do Redis, explorando al√©m do uso b√°sico como cache. Vamos focar em como o Redis pode ser usado como um banco de dados completo e vers√°til.


Para uma experi√™ncia premium, como a que eu quero que voc√™s tenham, recomendo MUITO utilizar o Redis Insight (App ou Web) pra apoiar na visualiza√ß√£o dos dados.

https://redis.com/redis-enterprise/redis-insight/

---

## ü§Ø N√£o tem um Redis ainda? Agora voc√™ vai ter! üöÄ

O **Redis** √© sempre o mesmo, independente da offering dele. Jamais iremos criar imped√¢ncia ou complexidade entre as APIs, e isso inclui o Community Edition.

De qualquer forma, recomendo que voc√™s fa√ßam esse tutorial usando o [Redis Cloud](https://redis.io/try-free/). Free forever, sem pegadinha, e voc√™ j√° faz o onboarding na plataforma SaaS.



## Objetivos do Workshop

O objetivo deste notebook √© apresentar as principais estruturas de dados do Redis e proporcionar uma pr√°tica interativa com cada uma delas. Nosso foco ser√° na execu√ß√£o de comandos e na manipula√ß√£o de dados em tempo real, com exemplos que voc√™ pode adaptar para suas pr√≥prias aplica√ß√µes.

**Nota**: J√° configuramos um ambiente de desenvolvimento no Google Colab para voc√™ come√ßar imediatamente.

## Setup Inicial

Antes de come√ßar a explorar, vamos configurar o ambiente e garantir que voc√™ esteja pronto para executar comandos Redis diretamente no Google Colab.

In [ ]:
# Instala a biblioteca Redis para Python
!pip install -q redis

# Instala a CLI do Redis (redis-tools) para executar comandos diretamente
!apt-get update -qq
!apt-get install -y -qq redis-tools

# Configura√ß√£o da conex√£o com o Redis Cloud (preencha com suas credenciais)
import os

REDIS_HOST = "redis-18443.c309.us-east-2-1.ec2.redns.redis-cloud.com"  # Exemplo: "redis-12345.c1.sa-east-1-3.ec2.cloud.redislabs.com"
REDIS_PORT = 18443             # Porta do seu Redis
REDIS_PASSWORD = "Secret@42"   # Senha do Redis

# Definindo a vari√°vel de ambiente para usar a redis-cli
os.environ["REDIS_CONN"] = f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"

# Testando a conex√£o com o Redis Cloud
!redis-cli $REDIS_CONN PING

# Importando a biblioteca redis-py e testando a conex√£o
import redis

r = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD,
    decode_responses=True
)

# Testando a conex√£o com o Redis usando redis-py
if r.ping():
    print("Conex√£o com o Redis bem-sucedida!")
else:
    print("Erro ao conectar com o Redis.")

Agora que voc√™ est√° com tudo configurado, vamos come√ßar a explorar os data types b√°sicos do Redis e realizar alguns exerc√≠cios interativos para entender como eles funcionam na pr√°tica.

# Data Types B√°sicos no Redis

**O Redis oferece uma variedade de estruturas de dados, cada uma otimizada para diferentes cen√°rios de uso. Aqui, vamos explorar os principais data types com exemplos r√°pidos para entender suas funcionalidades.**

## 1. Strings
As **Strings** s√£o o data type mais simples no Redis, armazenando qualquer sequ√™ncia de bytes, como texto ou n√∫meros. Elas s√£o amplamente usadas para contadores, sess√µes de usu√°rio, e caching de valores simples.

**Exemplo de Caso de Uso**: Cache Aside, contadores e KV-stores, Lock Distribu√≠do, Rate Limiter, Session Storage, Feature Flags, Gaming, etc.

### Exemplo com Strings



In [ ]:
# Armazenar um valor simples (pode ser uma configura√ß√£o ou dado de cache)
r.set("config:modo", "produ√ß√£o")
print("Modo atual:", r.get("config:modo"))

# Atualizar o valor (simular uma mudan√ßa de configura√ß√£o)
r.set("config:modo", "manuten√ß√£o")
print("Modo atualizado:", r.get("config:modo"))

# Usar Strings como um contador para likes em um post
r.set("post:123:likes", 10)
print("Likes iniciais:", r.get("post:123:likes"))

# Incrementar e decrementar o contador de likes
r.incr("post:123:likes")
print("Likes ap√≥s um incremento:", r.get("post:123:likes"))
r.decr("post:123:likes")
print("Likes ap√≥s um decremento:", r.get("post:123:likes"))

# Simular uma expira√ß√£o (TTL) para uma sess√£o de usu√°rio
r.setex("sessao:usuario:456", 30, "ativo")
print("Sess√£o de usu√°rio criada com TTL de 30 segundos.")

## 2. Lists

As **Lists** s√£o cole√ß√µes ordenadas de strings, √∫teis para filas de tarefas ou logs. Podem funcionar como uma fila (FIFO) ou uma pilha (LIFO). A API do Redis fornece muitos comandos seguros, at√¥micos e previs√≠veis para voc√™ implementar arquiteturas e filas robustas e prontas para PROD. Simples √© bem diferente de fr√°gil. 90% de todas as transa√ß√µes financeiras que voc√™ faz em um dia passaram por um Redis.

Caso de Uso: Implementar filas simples de mensagens, arquiteturas desacopladas, arquiteturas ass√≠ncronas, sistemas de vaz√£o, integra√ß√£o com Lambdas, Triggers, Filas de Trabalho, etc.

### Exemplo com Lists


In [ ]:
# Simular uma fila de tarefas com LPUSH e RPOP (FIFO)
r.lpush("fila:tarefas", "tarefa1", "tarefa2", "tarefa3")
print("Tarefas na fila (FIFO):", r.lrange("fila:tarefas", 0, -1))

# Remover e processar a tarefa mais antiga
tarefa = r.rpop("fila:tarefas")
print("Processando:", tarefa)
print("Tarefas restantes na fila:", r.lrange("fila:tarefas", 0, -1))

# Simular uma pilha de mensagens (LIFO)
r.rpush("pilha:mensagens", "mensagem1", "mensagem2", "mensagem3")
print("Mensagens na pilha (LIFO):", r.lrange("pilha:mensagens", 0, -1))

# Remover e processar a mensagem mais recente
mensagem = r.rpop("pilha:mensagens")
print("Processando mensagem:", mensagem)
print("Mensagens restantes na pilha:", r.lrange("pilha:mensagens", 0, -1))

## 3. Sets

Os **Sets** s√£o cole√ß√µes de strings √∫nicas, ideais para armazenar elementos sem duplicatas. S√£o √∫teis para deduplica√ß√£o, gerenciamento de IDs √∫nicos, ou at√© para opera√ß√µes de conjuntos, como interse√ß√£o e uni√£o.

Caso de Uso: Listar usu√°rios √∫nicos online em uma aplica√ß√£o, deduplicar itens, aplicar teoria dos conjuntos, ou at√© criar √≠ndices secund√°rios personalizados.

### Exemplo com Sets

In [ ]:
# Adicionar usu√°rios √∫nicos online
r.sadd("usuarios:online", "alice", "bob", "carol")
print("Usu√°rios online:", r.smembers("usuarios:online"))

# Adicionar um usu√°rio duplicado (n√£o ser√° adicionado novamente)
r.sadd("usuarios:online", "alice")
print("Usu√°rios online (ap√≥s tentativa de duplica√ß√£o):", r.smembers("usuarios:online"))

# Remover um usu√°rio que saiu
r.srem("usuarios:online", "bob")
print("Usu√°rios online (ap√≥s sa√≠da de Bob):", r.smembers("usuarios:online"))

# Opera√ß√µes de conjuntos: Simular interse√ß√£o com outro set de usu√°rios
r.sadd("usuarios:ativos", "alice", "dave", "carol")
interseccao = r.sinter("usuarios:online", "usuarios:ativos")
print("Usu√°rios online e ativos:", interseccao)

## 4. Sorted Sets

Os **Sorted Sets** s√£o como sets, mas com uma pontua√ß√£o associada a cada elemento, o que permite manter os elementos ordenados. Eles s√£o ideais para rankings, agendamentos, e qualquer caso em que a ordem baseada em pontua√ß√£o seja importante.

Caso de Uso: Classifica√ß√£o de jogadores em um jogo, leaderboards em geral, an√°lises em tempo real, agendamento de tarefas, ou filas de prioridade.

### Exemplo com Sorted Sets



In [ ]:
# Adicionar jogadores com suas pontua√ß√µes
r.zadd("jogo:ranking", {"player1": 1500, "player2": 3000, "player3": 2500})
print("Ranking dos jogadores (do menor para o maior):", r.zrange("jogo:ranking", 0, -1, withscores=True))

# Consultar o ranking em ordem decrescente (do maior para o menor)
print("Ranking dos jogadores (do maior para o menor):", r.zrevrange("jogo:ranking", 0, -1, withscores=True))

# Atualizar a pontua√ß√£o de um jogador
r.zincrby("jogo:ranking", 500, "player1")
print("Ranking atualizado (do maior para o menor):", r.zrevrange("jogo:ranking", 0, -1, withscores=True))

# Consultar a posi√ß√£o de um jogador
posicao = r.zrevrank("jogo:ranking", "player1")
print("Posi√ß√£o do player1 no ranking:", posicao + 1)  # +1 para uma posi√ß√£o humana (come√ßando do 1)

# Remover um jogador do ranking
r.zrem("jogo:ranking", "player2")
print("Ranking ap√≥s remo√ß√£o de player2:", r.zrevrange("jogo:ranking", 0, -1, withscores=True))

## 5. Hashes

Os **Hashes** s√£o cole√ß√µes de pares campo-valor, √∫teis para armazenar objetos, como perfis de usu√°rios ou configura√ß√µes. Voc√™ tamb√©m pode setar TTLs para cada sub-key, abrindo ainda mais possibilidades e controle.

Caso de Uso: Armazenar informa√ß√µes de usu√°rio de maneira estruturada, banco de dados prim√°rio, NoSQL, ORM, Session Storage, AIML Feature Stores, etc.

### Exemplo com Hashes

In [ ]:
# Criar um hash com informa√ß√µes de usu√°rio
r.hset("usuario:1001", mapping={"nome": "Gabriel", "email": "gabriel@example.com", "idade": "32"})
print("Perfil do usu√°rio:", r.hgetall("usuario:1001"))

# Recuperar apenas um campo espec√≠fico (email)
email = r.hget("usuario:1001", "email")
print("Email do usu√°rio:", email)

# Atualizar a idade do usu√°rio
r.hset("usuario:1001", "idade", "33")
print("Perfil atualizado:", r.hgetall("usuario:1001"))

# Remover um campo do hash (por exemplo, o email)
r.hdel("usuario:1001", "email")
print("Perfil ap√≥s remo√ß√£o do email:", r.hgetall("usuario:1001"))

# Definir um TTL para a chave do usu√°rio (expira ap√≥s 60 segundos)
r.expire("usuario:1001", 60)
print("TTL definido para a chave 'usuario:1001' (60 segundos)")

### Exerc√≠cio com Hashes - HASH Sub-Key Expiration

Com o Redis 7.4, √© poss√≠vel aplicar expira√ß√µes (TTL) a campos individuais, o que abre novas possibilidades para gerenciar dados tempor√°rios.

In [ ]:
# Criar um hash simulando dados de sess√£o do usu√°rio
r.hset("sessao:usuario:1337", mapping={
    "nome": "Gabriel",
    "sobrenome": "Cerioni",
    "token": "super-token-secreto<....>",            # Token que expira
    "ultimo_login": "2024-11-04",
    "pais_origem": "BRA",
    "geoloc_estimado": "-46.63389,-23.55028",        # Pra√ßa da S√©, SP
    "user_tier_level": 3                             # Imagine um sistema de Tier de 0-10 (noob to VIP)
})
print("Dados da sess√£o:", r.hgetall("sessao:usuario:1337"))

# Aplicar um TTL de 30 segundos apenas ao campo 'token' usando HEXPIRE
r.execute_command("HEXPIRE", "sessao:usuario:1337", 30, "FIELDS", 1, "token")
print("TTL de 30 segundos definido para o campo 'token'.")

# Verificar os dados da sess√£o (o token ainda est√° presente)
print("Dados da sess√£o (ap√≥s definir TTL):")
for campo, valor in r.hgetall("sessao:usuario:1337").items():
    print(f"{campo}: {valor}")

# Ap√≥s 30 segundos, o campo 'token' ser√° automaticamente removido. Aguarde para ver o efeito.

# OPCIONAL - Remover o TTL do campo 'token', tornando-o persistente novamente
#r.execute_command("HPERSIST", "sessao:usuario:1337", "FIELDS", 1, "token")
#print("O campo 'token' agora √© persistente novamente:", r.hgetall("sessao:usuario:1337"))

### Teaser de Indexa√ß√£o com RediSearch
#### *Calma, voc√™ ainda vai aprender a usar o `Redis Search and Query Engine`!*

Agora que voc√™ aprendeu a manipular dados estruturados com Hashes, imagine o poder de realizar buscas otimizadas e complexas. No **Redis**, com **RediSearch**, √© poss√≠vel indexar campos de forma poderosa e eficiente, como neste exemplo r√°pido de indexa√ß√£o e busca que veremos mais a fundo na pr√≥xima sess√£o:





In [ ]:
from redis.commands.search.field import TagField, NumericField, GeoField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

# 1. Deletar o √≠ndice se ele j√° existir, para garantir que come√ßamos do zero
try:
    r.ft("idx:sessao_usuarios").dropindex(delete_documents=False)
except:
    pass

# 2. Criar o √≠ndice para o hash 'sessao:usuario:'
r.ft("idx:sessao_usuarios").create_index(
    [
        TagField("pais_origem"),
        NumericField("user_tier_level"),
        GeoField("geoloc_estimado")
    ],
    definition=IndexDefinition(prefix=["sessao:usuario:"], index_type=IndexType.HASH)
)

# 3. Indexar um hash de exemplo com dados de sess√£o do usu√°rio
r.hset("sessao:usuario:1337", mapping={
    "nome": "Gabriel",
    "sobrenome": "Cerioni",
    "token": "super-token-secreto<....>",
    "ultimo_login": "2024-11-04",
    "pais_origem": "BRA",
    "geoloc_estimado": "-46.63389,-23.55028",  # Pra√ßa da S√©, SP
    "user_tier_level": 3
})
print("‚úÖ Dados de sess√£o indexados com sucesso!")

# 4. Executar uma busca √∫nica: usu√°rios do Brasil, n√≠vel de tier entre 2 e 5, pr√≥ximos ao centro de SP
query = Query("@pais_origem:{BRA} @user_tier_level:[2 5] @geoloc_estimado:[-46.63389 -23.55028 5 km]")

# 5. Executar a busca e exibir os resultados
res = r.ft("idx:sessao_usuarios").search(query)
print("üîç Resultado da busca com RediSearch:")
for doc in res.docs:
    print(doc.__dict__)

# Data Types Avan√ßados no Redis

Agora que voc√™ j√° conhece os tipos de dados b√°sicos no Redis, chegou a hora de explorar algumas das funcionalidades mais poderosas e avan√ßadas. Esses data types s√£o perfeitos para cen√°rios que exigem alta performance, escalabilidade e funcionalidades que v√£o muito al√©m de simples caches.

Prepare-se para ver o Redis como um verdadeiro *Canivete Su√≠√ßo* de solu√ß√µes de dados, desde mensageria em tempo real at√© armazenamento eficiente de documentos JSON, e tudo isso com exemplos pr√°ticos que voc√™ poder√° adaptar para seus projetos.

## RedisJSON - Redis como um NoSQL Prim√°rio

O **RedisJSON** √© um m√≥dulo que permite armazenar, consultar e modificar documentos JSON diretamente no Redis. Ele √© especialmente √∫til para quem precisa trabalhar com dados hier√°rquicos e quer a flexibilidade de documentos JSON, mas com a performance do Redis.

Ele √© um bom candidato a ser o seu NoSQL Prim√°rio. Integra-se nativamente com ferramentais de CDC e ORM, onde sua aplica√ß√£o considera o Redis como um DAO Repository ou algo similar, assim como qualquer outro banco de dados NoSQL.

### Caso de Uso: Cat√°logo de Produtos

Imagine que voc√™ esteja criando um cat√°logo de produtos para um e-commerce. Voc√™ pode armazenar informa√ß√µes complexas como detalhes do produto, pre√ßo, invent√°rio e reviews em formato JSON. Isso permite consultas eficientes e a capacidade de atualizar dados espec√≠ficos de maneira at√¥mica.

In [ ]:
# Importando a biblioteca para trabalhar com JSON no Redis
import redis
from redis.commands.json.path import Path

# Conectando ao Redis (ajuste as credenciais conforme necess√°rio)
r = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD,
    decode_responses=True
)

# 1. Criar um documento JSON para um produto no cat√°logo
produto = {
    "id": "123",
    "nome": "Fone de Ouvido Bluetooth",
    "marca": "MarcaX",
    "preco": 299.99,
    "estoque": 50,
    "reviews": [
        {"usuario": "Alice", "comentario": "√ìtimo produto!", "nota": 5},
        {"usuario": "Bob", "comentario": "Bom custo-benef√≠cio.", "nota": 4}
    ]
}

# 2. Armazenar o documento JSON no Redis
r.json().set("produto:123", Path.root_path(), produto)
print("‚úÖ Produto armazenado com sucesso!")

# 3. Consultar o nome e o pre√ßo do produto
nome = r.json().get("produto:123", Path(".nome"))
preco = r.json().get("produto:123", Path(".preco"))
print(f"Nome: {nome}, Pre√ßo: R${preco}")

# 4. Atualizar o pre√ßo do produto
r.json().set("produto:123", Path(".preco"), 279.99)
print("üí∞ Pre√ßo atualizado para R$279.99!")

# 5. Adicionar um novo review ao produto
novo_review = {"usuario": "Carlos", "comentario": "Funciona muito bem!", "nota": 5}
r.json().arrappend("produto:123", Path(".reviews"), novo_review)
print("üìù Novo review adicionado!")

# 6. Consultar o documento atualizado
produto_atualizado = r.json().get("produto:123")
print("üîç Produto atualizado:", produto_atualizado)

## Redis Pub/Sub - Mensageria em Tempo Real

O **Pub/Sub** do Redis √© uma ferramenta poderosa para sistemas de mensageria. Ele permite que aplica√ß√µes publiquem mensagens em canais espec√≠ficos e que outros sistemas ou servi√ßos se inscrevam (subscribe) para receber essas mensagens em tempo real. Isso √© muito √∫til para criar sistemas distribu√≠dos, notifica√ß√µes em tempo real, ou arquiteturas de eventos.

### Caso de Uso: Notifica√ß√µes em Tempo Real

Imagine que voc√™ tem um sistema de monitoramento em uma aplica√ß√£o de e-commerce, onde precisa alertar os usu√°rios sobre mudan√ßas de pre√ßo, eventos ou novas promo√ß√µes. O **Redis Pub/Sub** facilita a comunica√ß√£o em tempo real entre diferentes componentes do sistema.

### Exemplo com Pub/Sub

No exemplo abaixo, criaremos um simples sistema de publica√ß√£o e assinatura:

In [ ]:
import threading
import time

# Fun√ß√£o para o assinante
def subscriber():
    pubsub = r.pubsub()
    pubsub.subscribe("notificacoes")

    print("üì° Assinante aguardando mensagens no canal 'notificacoes'...")

    for mensagem in pubsub.listen():
        if mensagem["type"] == "message":
            print(f"üîî Nova mensagem recebida: {mensagem['data']}")

# Iniciar o assinante em uma thread separada
assinante_thread = threading.Thread(target=subscriber)
assinante_thread.start()

# Simular o publicador que envia mensagens
time.sleep(2)  # Simula um atraso antes de enviar a mensagem
r.publish("notificacoes", "Promo√ß√£o: 50% de desconto em todos os produtos!")
time.sleep(1)
r.publish("notificacoes", "Novidade: Lan√ßamos uma nova cole√ß√£o de ver√£o!")

## Redis Streams - Comunica√ß√£o Eficiente e Processamento de Dados

O **Redis Stream** √© uma estrutura de dados poderosa que permite capturar e distribuir eventos em tempo real. Ele √© √∫til em casos como processamento de logs, filas de eventos distribu√≠das, ou qualquer aplica√ß√£o que precise de alta disponibilidade e escalabilidade.

### Caso de Uso: Sistema de Processamento de Pedidos

Imagine um e-commerce processando pedidos em tempo real. Cada pedido pode ser registrado como uma mensagem na Stream, e grupos de consumidores podem processar esses pedidos em paralelo, distribuindo a carga de trabalho de maneira eficiente.

### Exemplo com Redis Streams

#### 1. Adicionando Mensagens √† Stream

Vamos criar uma Stream e adicionar mensagens que representam pedidos de um sistema de e-commerce.


In [ ]:
# 1. Adicionando Mensagens √† Stream
print("### Adicionando Mensagens √† Stream ###")
r.delete("pedidos_stream")  # Deletar a Stream se j√° existir

# Adicionar mensagens
r.xadd("pedidos_stream", {"pedido_id": "001", "cliente": "Gabs", "valor": "100.00"})
r.xadd("pedidos_stream", {"pedido_id": "002", "cliente": "Magro", "valor": "250.50"})
r.xadd("pedidos_stream", {"pedido_id": "003", "cliente": "Bart", "valor": "89.99"})

# Verificar mensagens na Stream
entradas = r.xrange("pedidos_stream", "-", "+")
print("Entradas na Stream de Pedidos:")
for entrada in entradas:
    print(f"ID: {entrada[0]}, Dados: {entrada[1]}")


#### 2. Criando um Grupo de Consumidores

Vamos criar um grupo de consumidores chamado grupo_pedidos para processar as mensagens.

In [ ]:
# 2. Criando um Grupo de Consumidores
print("\n### Criando um Grupo de Consumidores ###")
try:
    r.xgroup_create("pedidos_stream", "grupo_pedidos", id="0", mkstream=True)
    print("Grupo de consumidores 'grupo_pedidos' criado com sucesso!")
except redis.exceptions.ResponseError:
    print("Grupo de consumidores j√° existe, prosseguindo...")

#### 3. Consumindo Mensagens com um Consumidor

Vamos usar o `XREADGROUP` para consumir mensagens no grupo de consumidores.

In [ ]:
# 3. Consumindo Mensagens com um Consumidor
print("\n### Consumindo Mensagens com consumer1 ###")
entradas = r.xreadgroup("grupo_pedidos", "consumer1", {"pedidos_stream": ">"}, count=2, block=5000)

if entradas:
    print("Mensagens lidas pelo consumer1:")
    for stream, mensagens in entradas:
        for msg_id, dados in mensagens:
            print(f"ID: {msg_id}, Dados: {dados}")
            # Confirmar (acknowledge) o processamento da mensagem
            r.xack("pedidos_stream", "grupo_pedidos", msg_id)
            print(f"Mensagem {msg_id} reconhecida (acknowledged) pelo consumer1.")
else:
    print("Nenhuma mensagem dispon√≠vel para consumer1.")

#### 4. Gerenciamento de Mensagens Pendentes

Podemos verificar mensagens pendentes que ainda n√£o foram processadas e transferi-las para outro consumidor, se necess√°rio.


In [ ]:
# 4. Gerenciamento de Mensagens Pendentes
print("\n### Gerenciamento de Mensagens Pendentes ###")
pendentes = r.xpending("pedidos_stream", "grupo_pedidos")
print("Mensagens pendentes no grupo 'grupo_pedidos':", pendentes)

## RedisTimeSeries - Gerenciando S√©ries Temporais de Forma Eficiente

O **RedisTimeSeries** √© um m√≥dulo que permite coletar, armazenar, processar e consultar grandes volumes de dados de s√©ries temporais de forma eficiente. Ele √© ideal para aplica√ß√µes de monitoramento, m√©tricas de IoT, ou sistemas que precisam de dados hist√≥ricos com agrega√ß√µes em tempo real.

### Caso de Uso: Monitoramento de M√©tricas de Aplica√ß√µes (Estilo Prometheus)

Imagine que voc√™ queira monitorar o uso de CPU e mem√≥ria de um cluster de servidores em tempo real. **RedisTimeSeries** √© uma escolha ideal, pois permite armazenar essas m√©tricas e fazer consultas r√°pidas para an√°lise de tend√™ncias.

### Exemplo com RedisTimeSeries

#### 1. Criando S√©ries Temporais com Reten√ß√£o de Dados

Vamos come√ßar criando s√©ries temporais para monitorar m√©tricas de CPU e mem√≥ria, com um tempo de reten√ß√£o de dados de 30 dias.

In [ ]:
# Fun√ß√£o para criar uma s√©rie temporal de forma segura
def create_timeseries_if_not_exists(key, retention, labels):
    try:
        r.execute_command("TS.CREATE", key, "RETENTION", retention, "LABELS", *labels)
        print(f"S√©rie temporal '{key}' criada com sucesso!")
    except redis.exceptions.ResponseError as e:
        if "already exists" in str(e):
            print(f"S√©rie temporal '{key}' j√° existe, prosseguindo...")
        else:
            raise e

# Criar s√©ries temporais para CPU e mem√≥ria com 30 dias de reten√ß√£o (2.592.000.000 ms)
create_timeseries_if_not_exists("metric:cpu_usage", 2592000000, ["type", "cpu", "region", "east"])
create_timeseries_if_not_exists("metric:memory_usage", 2592000000, ["type", "memory", "region", "east"])

#### 2. Adicionando Dados em Tempo Real

Podemos adicionar dados de uso de CPU e mem√≥ria com a marca√ß√£o de tempo atual.

In [ ]:
# Adicionando dados com o timestamp atual
r.execute_command("TS.ADD", "metric:cpu_usage", "*", 75.5)  # 75.5% de uso de CPU
r.execute_command("TS.ADD", "metric:memory_usage", "*", 512)  # 512 MB de uso de mem√≥ria

print("Dados adicionados com sucesso √†s s√©ries temporais!")

#### 3. Consulta Simples: Recuperar Dados em Intervalo de Tempo

Vamos consultar o uso de CPU no √∫ltimo minuto.


In [ ]:
import time

# Recuperar o timestamp atual
current_time = int(time.time() * 1000)  # em milissegundos

# Consultar dados de uso de CPU nos √∫ltimos 60 segundos (60.000 ms)
cpu_usage = r.execute_command("TS.RANGE", "metric:cpu_usage", current_time - 60000, current_time)
print("Uso de CPU nos √∫ltimos 60 segundos:")
for data_point in cpu_usage:
    print(f"Timestamp: {data_point[0]}, Valor: {data_point[1]}")

#### 4. Agrega√ß√µes: M√©dia de Uso por Minuto

Para ver a m√©dia de uso de CPU a cada minuto, usamos agrega√ß√µes.

In [ ]:
# Agregar dados de uso de CPU a cada minuto (60.000 ms) com a m√©dia
cpu_avg = r.execute_command("TS.RANGE", "metric:cpu_usage", "-", "+", "AGGREGATION", "avg", 60000)
print("M√©dia de uso de CPU por minuto:")
for data_point in cpu_avg:
    print(f"Timestamp: {data_point[0]}, Valor: {data_point[1]}")

#### 5. Compacta√ß√£o de Dados

Podemos criar regras para compactar dados automaticamente, reduzindo o tamanho da s√©rie temporal.



In [ ]:
# Garantir que a s√©rie compactada n√£o exista antes de criar
try:
    r.execute_command("TS.CREATE", "metric:cpu_usage:avg_minute", "RETENTION", 0)  # Sem reten√ß√£o para a s√©rie compactada
    print("S√©rie compactada 'metric:cpu_usage:avg_minute' criada com sucesso.")
except redis.exceptions.ResponseError:
    print("A s√©rie 'metric:cpu_usage:avg_minute' j√° existe, prosseguindo...")

# Criar a regra de compacta√ß√£o de forma segura
try:
    r.execute_command("TS.CREATERULE", "metric:cpu_usage", "metric:cpu_usage:avg_minute", "AGGREGATION", "avg", 60000)
    print("Regra de compacta√ß√£o criada: uso de CPU ser√° agregado a cada minuto.")
except redis.exceptions.ResponseError:
    print("A regra de compacta√ß√£o j√° existe, prosseguindo...")

# Adicionar alguns dados para verificar a compacta√ß√£o
current_time = int(time.time() * 1000)  # Timestamp em milissegundos
r.execute_command("TS.ADD", "metric:cpu_usage", current_time, 60.0)
r.execute_command("TS.ADD", "metric:cpu_usage", current_time + 30000, 70.0)  # 30 segundos depois
r.execute_command("TS.ADD", "metric:cpu_usage", current_time + 60000, 80.0)  # 60 segundos depois

# Consultar os dados compactados imediatamente
cpu_avg_compacted = r.execute_command("TS.RANGE", "metric:cpu_usage:avg_minute", "-", "+")
print("Dados compactados de uso de CPU (m√©dia por minuto):")
for data_point in cpu_avg_compacted:
    print(f"Timestamp: {data_point[0]}, Valor: {data_point[1]}")

## Estruturas de Dados Probabil√≠sticas com Redis

O Redis oferece suporte a estruturas de dados probabil√≠sticas que economizam espa√ßo e s√£o extremamente r√°pidas para opera√ß√µes espec√≠ficas. Vamos explorar dois exemplos simples: **HyperLogLog** e **Bloom Filter**.

### 1. HyperLogLog

O **HyperLogLog** √© uma estrutura de dados probabil√≠stica que estima a cardinalidade (n√∫mero de elementos √∫nicos) de um conjunto. Em vez de armazenar todos os elementos, ele fornece uma estimativa com um erro padr√£o de `0,81%`, usando apenas at√© `12 KB` de mem√≥ria.

**Casos de Uso:** Contar visitantes √∫nicos de um site, motores de fraude, consultas de pesquisa √∫nicas ou qualquer aplica√ß√£o que necessite de contagem de elementos √∫nicos de forma eficiente, com alt√≠ssima performance e baixo custo.

#### Exemplo Simples com HyperLogLog



In [ ]:
# Adicionar elementos a um HyperLogLog
r.pfadd("visitors", "Alice", "Bob", "Charlie", "Alice")
print(r.pfcount("visitors"))  # Sa√≠da: 3 (estimativa de visitantes √∫nicos)

# Adicionar mais visitantes
r.pfadd("visitors", "Dave", "Eve")
print(r.pfcount("visitors"))  # Sa√≠da: 5 (estimativa atualizada)

# Unir HyperLogLogs (exemplo de caso de uso com m√∫ltiplas origens de visitantes)
r.pfadd("new_visitors", "Frank", "Grace")
r.pfmerge("all_visitors", "visitors", "new_visitors")
print(r.pfcount("all_visitors"))  # Sa√≠da: estimativa total

## 2. Bloom Filter

O **Bloom Filter** √© uma estrutura de dados probabil√≠stica que verifica se um elemento pertence a um conjunto, mas com possibilidade de falsos positivos (afirma√ß√£o errada de presen√ßa) e garantia de aus√™ncia correta. √â ideal para casos onde queremos evitar opera√ß√µes custosas ao verificar a presen√ßa de um elemento.

**Casos de uso:** Detec√ß√£o de fraudes financeiras, verifica√ß√£o r√°pida de nomes de usu√°rios, evitar a exibi√ß√£o repetida de an√∫ncios ou evitar a duplica√ß√£o de nomes de modelos de produtos.

### Exemplo Simples com Bloom Filter



In [ ]:
# Tentar criar o Bloom Filter somente se ele n√£o existir
try:
    r.bf().reserve("product_filter", 0.001, 1000)
    print("Bloom Filter 'product_filter' criado com sucesso!")
except redis.exceptions.ResponseError as e:
    if "item exists" in str(e):
        print("Bloom Filter 'product_filter' j√° existe, prosseguindo...")
    else:
        raise e

# Adicionar produtos ao filtro
r.bf().add("product_filter", "Bike Model X")
r.bf().add("product_filter", "Bike Model Y")

# Verificar se os produtos est√£o no filtro
print(r.bf().exists("product_filter", "Bike Model X"))  # Sa√≠da: True
print(r.bf().exists("product_filter", "Bike Model Z"))  # Sa√≠da: False

# Adicionar m√∫ltiplos produtos e verificar a exist√™ncia
r.bf().madd("product_filter", "Bike Model Z", "Bike Model W")
print(r.bf().mexists("product_filter", "Bike Model Z", "Bike Model W"))  # Sa√≠da: [True, True]